In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
test_dir = '/opt/ml/input/data/eval'

In [3]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [4]:
def convert(x):
    if x == 0:
        return '0'
    elif x == 1:
        return '1'
    elif x == 2:
        return '10'
    elif x == 3:
        return '11'
    elif x == 4:
        return '12'
    elif x == 5:
        return '13'
    elif x == 6:
        return '14'
    elif x == 7:
        return '15'
    elif x == 8:
        return '16'
    elif x == 9:
        return '17'
    elif x == 10:
        return '2'
    elif x == 11:
        return '3'
    elif x == 12:
        return '4'
    elif x == 13:
        return '5'
    elif x == 14:
        return '6'
    elif x == 15:
        return '7'
    elif x == 16:
        return '8'
    elif x == 17:
        return '9'

In [5]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

device = torch.device('cuda')

model_1 = torch.load("/opt/ml/model/model_data_1.pt")
model_1.eval()
# model_2 = torch.load("/opt/ml/model/model_data_2.pt")
# model_2.eval()
# model_3 = torch.load("/opt/ml/model/model_data_3.pt")
# model_3.eval()
# model_4 = torch.load("/opt/ml/model/model_data_4.pt")
# model_4.eval()
# model_5 = torch.load("/opt/ml/model/model_data_5.pt")
# model_5.eval()

all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        # pred = model_conv(images)
        pred_1 = model_1(images)
        # pred_2 = model_2(images)
        # pred_3 = model_3(images)
        # pred_4 = model_4(images)
        # pred_5 = model_5(images)
        pred = pred_1
        # pred = pred_1 + pred_2 + pred_3 + pred_4 + pred_5
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
        
submission['ans'] = all_predictions

submission['ans'] = submission['ans'].apply(lambda x : convert(x))

submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
